In [1]:
import pandas as pd
import numpy as np
from random import sample
from pandas import *
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Libraries from Alan
from __future__ import division
import json
import os
from matplotlib.pyplot import *

import requests
#import tmy_ids

# Show Plots in the Notebook
%matplotlib inline

# Make Plots larger
rcParams['figure.figsize']= (10, 6)
rcParams['font.size'] = 14

%matplotlib inline

print 'Pandas version: ' + pd.__version__

Pandas version: 0.14.0


## This notebook combines the data from other notebooks into a file formatted like the "Monthly Weather Item" table in the AKWarm energy library

### Column Descriptions:
 
- DBTempAvg: The average monthly dry-bulb temperature in degrees Fahrenheit
- RHum (%): The average monthly relative humidity
- Wind: Average monthly wind speed in miles per hour
- South_Vert_Solar through North_Vert_Solar: The average monthly solar irradiation hitting a vertical surface facing South/East/West/North expressed in Btus/ft2/day.
- HorizSol: The average monthly solar irradiation hitting a horizontal surface expressed in Btus/ft2/day.
- bin_data:  A string that provides warm-weather climate information  in bin form that is used by AKWarm-Commercial to more accurately calculate cooling loads.  Each bin is separated by a space and information within the one bin is separated by commas. An example is:  '71,23,2.560 73,14,2.923' which gives information for two bins.  For each bin, the first value is the midpoint temperature of the bin, e.g. for a bin width of 2 deg F, a value of 71 means the bin is 70 F <= temperature < 72 F.  The bin data starts at 50 degrees and continues upwards. The second number describing the bin is the number of hours that fall into the indicated bin.  The third number describing the bin is the ratio of solar power (W or Btu/hour) occuring for the bin  hours relative to the average solar power for the time period analyzed, averaging all of the hours in the time period, including those that fall outside of the bins.


In [3]:
wind = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\Wind_for_AKWarm_Lib2.csv")
wind.head()

,Unnamed: 0,station_name,MONTH,Wind
0,0,ALYESKA AK US,1,6.001083
1,1,ALYESKA AK US,2,6.069240
2,2,ALYESKA AK US,3,6.586587
3,3,ALYESKA AK US,4,6.633873
4,4,ALYESKA AK US,5,7.532658
5,5,ALYESKA AK US,6,7.468497
6,6,ALYESKA AK US,7,6.690240
7,7,ALYESKA AK US,8,6.212169
8,8,ALYESKA AK US,9,6.344874
9,9,ALYESKA AK US,10,6.106086


In [4]:
solar = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\20_Solar_Sites_For_AKWarm.csv")
solar.head()

,Unnamed: 0,station_name,month,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
0,0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1,243.439017,72.790127,70.255741,40.922696,90.816917
1,1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,2,836.873376,284.696415,299.806488,116.571676,340.273159
2,2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,3,986.226933,503.609820,532.198766,230.233863,705.361790
3,3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,4,1120.400031,789.046156,747.676149,354.949878,1118.684836
4,4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,5,1027.195038,929.611070,893.778646,519.205526,1421.677782


In [5]:
rh = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\relative_humidity_for_akwarm_library.csv")
rh.head()

,Unnamed: 0,STATION_NAME,MONTH,RHum (%)
0,0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1,75.784946
1,1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,2,77.026786
2,2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,3,60.801075
3,3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,4,64.655556
4,4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,5,61.629032


In [8]:
bin_data = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\AKWarm_Bin_Data.csv", header=None, names=['station_name', 'month', 'bin_data'])
bin_data.head()

,station_name,month,bin_data
0,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,1,NaN
1,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,2,NaN
2,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,3,NaN
3,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,4,"51,10,1.586 53,1,2.708 55,1,3.969"
4,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT AK US,5,"51,171,1.226 53,81,1.977 55,43,2.203 57,32,2.4..."


In [9]:
temps = pd.read_csv(r"C:\Users\dustin\Documents\IPython Notebooks\Main_Weather_Cities_Monthly_Temps.csv")
temps.head()

,Unnamed: 0,STATION_NAME,MONTH,DBTempAvg
0,144,COLLEGE 5 NW AK US,1,1.0
1,145,COLLEGE 5 NW AK US,2,6.3
2,146,COLLEGE 5 NW AK US,3,15.5
3,147,COLLEGE 5 NW AK US,4,32.8
4,148,COLLEGE 5 NW AK US,5,48.5


In [13]:
wind_and_solar = pd.merge(wind, solar, how='left', left_on=['station_name', 'MONTH'], right_on=['station_name', 'month'])
wind_and_solar.head()

,Unnamed: 0_x,station_name,MONTH,Wind,Unnamed: 0_y,month,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
0,0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
wind_and_solar = wind_and_solar.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'month'], 1)
wind_and_solar.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN


In [15]:
wind_and_solar.describe()

,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
count,624.000000,624.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,6.500000,7.730152,746.979498,574.889605,483.082864,289.840067,734.430805
std,3.454822,3.248982,356.002630,385.032819,354.488693,226.998103,547.173316
min,1.000000,1.626293,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.750000,5.446537,463.781877,238.602572,148.168851,83.206714,193.012270
50%,6.500000,7.178110,765.939112,605.380183,459.302673,240.049803,680.798376
75%,9.250000,9.524075,992.970773,846.004007,799.393005,480.477987,1247.320899
max,12.000000,18.781300,1744.176559,1523.454968,1323.945180,844.142030,1795.547118


In [16]:
wind_solar_rh = pd.merge(wind_and_solar, rh, how='left', left_on=['station_name', 'MONTH'], right_on=['STATION_NAME', 'MONTH'])
wind_solar_rh.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,Unnamed: 0,STATION_NAME,RHum (%)
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
wind_solar_rh = wind_solar_rh.drop(['Unnamed: 0', 'STATION_NAME'], 1)
wind_solar_rh.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,RHum (%)
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
wind_solar_rh_bin = pd.merge(wind_solar_rh, bin_data, how='left', left_on=['station_name', 'MONTH'], right_on=['station_name', 'month'])
wind_solar_rh_bin.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,RHum (%),month,bin_data
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
wind_solar_rh_bin = wind_solar_rh_bin.drop(['month'], 1)
wind_solar_rh_bin.describe()

ValueError: labels ['month'] not contained in axis

In [22]:
wind_solar_rh_bin.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,RHum (%),bin_data
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
main_weather_city_library = pd.merge(wind_solar_rh_bin, temps, how='left', left_on=['station_name', 'MONTH'], right_on=['STATION_NAME', 'MONTH'])
main_weather_city_library.head()

,station_name,MONTH,Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,RHum (%),bin_data,Unnamed: 0,STATION_NAME,DBTempAvg
0,ALYESKA AK US,1,6.001083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1572,ALYESKA AK US,22.0
1,ALYESKA AK US,2,6.069240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1573,ALYESKA AK US,23.9
2,ALYESKA AK US,3,6.586587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1574,ALYESKA AK US,27.9
3,ALYESKA AK US,4,6.633873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1575,ALYESKA AK US,36.0
4,ALYESKA AK US,5,7.532658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1576,ALYESKA AK US,45.3


In [24]:
list(main_weather_city_library.columns.values)

['station_name',
 'MONTH',
 'Wind',
 'South_Vert_Solar',
 'East_Vert_Solar',
 'West_Vert_Solar',
 'North_Vert_Solar',
 'HorizSol',
 'RHum (%)',
 'bin_data',
 'Unnamed: 0',
 'STATION_NAME',
 'DBTempAvg']

In [25]:
main_weather_city_library = main_weather_city_library[['station_name', 'MONTH', 'DBTempAvg', 'RHum (%)', 'Wind', 'South_Vert_Solar',
 'East_Vert_Solar', 'West_Vert_Solar', 'North_Vert_Solar', 'HorizSol', 'bin_data']]
main_weather_city_library.head()

,station_name,MONTH,DBTempAvg,RHum (%),Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol,bin_data
0,ALYESKA AK US,1,22.0,NaN,6.001083,NaN,NaN,NaN,NaN,NaN,NaN
1,ALYESKA AK US,2,23.9,NaN,6.069240,NaN,NaN,NaN,NaN,NaN,NaN
2,ALYESKA AK US,3,27.9,NaN,6.586587,NaN,NaN,NaN,NaN,NaN,NaN
3,ALYESKA AK US,4,36.0,NaN,6.633873,NaN,NaN,NaN,NaN,NaN,NaN
4,ALYESKA AK US,5,45.3,NaN,7.532658,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
main_weather_city_library.describe()

,MONTH,DBTempAvg,RHum (%),Wind,South_Vert_Solar,East_Vert_Solar,West_Vert_Solar,North_Vert_Solar,HorizSol
count,624.000000,624.000000,384.000000,624.000000,240.000000,240.000000,240.000000,240.000000,240.000000
mean,6.500000,33.626122,75.288430,7.730152,746.979498,574.889605,483.082864,289.840067,734.430805
std,3.454822,18.957268,9.101184,3.248982,356.002630,385.032819,354.488693,226.998103,547.173316
min,1.000000,-16.100000,45.926075,1.626293,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.750000,22.975000,69.671965,5.446537,463.781877,238.602572,148.168851,83.206714,193.012270
50%,6.500000,35.950000,76.731746,7.178110,765.939112,605.380183,459.302673,240.049803,680.798376
75%,9.250000,49.400000,81.685820,9.524075,992.970773,846.004007,799.393005,480.477987,1247.320899
max,12.000000,62.500000,96.970430,18.781300,1744.176559,1523.454968,1323.945180,844.142030,1795.547118


In [27]:
main_weather_city_library.to_csv("main_weather_city_library.csv")